In [1]:
import datetime
import pandas as pd
import numpy as np
import random
from scipy import sparse
from scipy.special import digamma
from scipy.sparse import csr_matrix
from time import time
from functools import reduce
# from guppy import hpy

K = 9
numIter = 100

path = r"D:/Download/data/"
# datasetName = "lastfm_sample_190617/"
datasetName = r"\go_sf"
trainFile = path + datasetName + r"\train.csv"
validationFile = path + datasetName + r"\validation.csv"
testFile = path + datasetName + r"\test.csv"
train = pd.read_csv(trainFile, header=None)
validation = pd.read_csv(validationFile, header=None)
test = pd.read_csv(testFile, header=None)

train.columns = ["user_id", "loc_id", "count"]
validation.columns = ["user_id", "loc_id", "count"]
test.columns = ["user_id", "loc_id", "count"]

def function01(data, train):
	array = []
	for index, row in data.iterrows():
		temp = [[row['user_id'], row['loc_id'], train]] * row['count']
		array.extend(temp)

	df = pd.DataFrame(array)
	# df = df.sample(frac=0.4, replace=True, random_state=1)
	df.columns = ["user_id", "loc_id", "train"]
	return df

train_df = function01(train,1)
validation_df = function01(validation,1)
test_df = function01(test, 0)
# train_user_set = set(train_df['user_id'].tolist())
# validation_user_set = set(validation_df['user_id'].tolist())
# user_of_interest = train_user_set.intersection(validation_user_set)


# train_df = train_df[train_df['user_id'].isin(user_of_interest)]
# validation_df = validation_df[validation_df['user_id'].isin(user_of_interest)]
# test_df = test_df[test_df['user_id'].isin(user_of_interest)]

train_df = pd.concat([train_df,validation_df])
test_df = test_df[test_df['loc_id'].isin(train_df['loc_id'].tolist())]

df = pd.concat([train_df, test_df])

loc_df = pd.DataFrame(df['loc_id'].unique())
loc_df['index1'] = loc_df.index
loc_df.columns = ['loc_id', 'index1']

df = pd.merge(df, loc_df, how='inner', left_on=['loc_id'], right_on=['loc_id']) \
	[['user_id', 'index1', 'train']]
df.columns = ['user_id', 'loc_id', 'train']
df = df.sample(frac=1, random_state=123).sort_values(by=['user_id', 'train'], ascending=[1, 0])

# print(df[df['user_id'] == 100])
num_record = len(df)
print(num_record)


137474


In [15]:
g = df.groupby(['loc_id'])['train'].apply(lambda x: x.count()).sort_index()
# g = df.groupby(['loc_id'])['user_id'].apply(np.unique).apply(lambda x: len(x)).sort_index()
g = np.array(g.tolist())
g = g.T
g = g / float(sum(g))
g

array([6.18298733e-04, 9.31085151e-04, 2.61867699e-04, ...,
       2.18223082e-05, 2.90964110e-05, 9.45633356e-05])